In [1]:
import numpy as np
import pandas as pd
import jieba
import string
from fastText import load_model
import platform
platform.system()

'Darwin'

In [2]:
cwe = np.load("zh.bin.syn0.npy")

In [4]:
cwe.shape

(50101, 300)

In [5]:
cwe_neg = np.load("zh.bin.syn1neg.npy")

In [6]:
cwe_neg.shape

(50101, 300)

# Word Embedding Matrix Lookup (cross platform)
Reference: [https://github.com/facebookresearch/fastText/tree/master/python](https://github.com/facebookresearch/fastText/tree/master/python)

In [2]:
if platform.system() == 'Windows':
    with open("zh/wiki.zh.vec", "r", encoding='utf-8') as fin:
        words = fin.readlines()
elif platform.system() == 'Darwin':  # Mac OSX
    with open("zh/wiki.zh.vec", "r") as fin:
        words = fin.readlines()

In [3]:
len(words)

332648

In [4]:
if platform.system() == 'Windows':
    wiki_vocab = [x.split(' ')[0] for x in words[1:]]
elif platform.system() == 'Darwin':
    wiki_vocab = [x.split(' ')[0].decode('utf-8') for x in words[1:]]

In [5]:
len(wiki_vocab)

332647

In [6]:
print(wiki_vocab[100])

日本


In [ ]:
# wiki_embeddings = np.array([x.strip().split(' ')[1:] for x in words[1:]])  # will probably run out of memory

In [2]:
def read_vocab(file_path):
    if platform.system() == 'Windows':
        with open(file_path, 'r', encoding='utf-8') as fin:
            vocab = [x.strip() for x in fin.readlines()]
            vocab = [x for x in vocab if x != '']
    elif platform.system() == 'Darwin':
        with open(file_path, 'r') as fin:
            vocab = [x.strip() for x in fin.readlines()]
            vocab = [x.decode('utf-8') for x in vocab if x != '']
    return vocab

In [3]:
vocab = read_vocab('../01_WebScraping/vocab.txt')

In [4]:
len(vocab)

9827

In [6]:
print(vocab[2])

与我无关


In [13]:
def custom_tokenizer(text):
    return [token for token in jieba.cut(text) if token not in string.punctuation+" " and token != '\n']

In [14]:
def read_lyrics(file_path):
    if platform.system() == 'Windows':
        lyrics = [line.strip() for line in open(file_path, 'r', encoding='utf-8').readlines()]
        lyrics = [custom_tokenizer(text) for text in lyrics]
    elif platform.system() == 'Darwin':
        lyrics = [line.strip() for line in open(file_path, 'r').readlines()]
        lyrics = [custom_tokenizer(text) for text in lyrics]
    return lyrics

In [15]:
lyrics = read_lyrics('../01_WebScraping/lyrics.txt')

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/45/9s5cv4kn143fggkxy4bdc89c0000gn/T/jieba.cache
DEBUG:jieba:Dumping model to file cache /var/folders/45/9s5cv4kn143fggkxy4bdc89c0000gn/T/jieba.cache
Loading model cost 3.991 seconds.
DEBUG:jieba:Loading model cost 3.991 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


In [16]:
for item in lyrics[:3]:
    print(len(item))

82
210
161


In [19]:
print(lyrics[:3][0])

[u'\u8fd9\u4e2a', u'\u4e16\u4e0a', u'\u672c', u'\u6ca1\u6709', u'\u8def', u'\u8d70', u'\u7684', u'\u4eba', u'\u591a', u'\u4e86', u'\u4e5f', u'\u5c31', u'\u6210', u'\u4e86', u'\u8def', u'\u7ecf\u8fc7', u'\u90a3\u4e9b', u'\u8ff7\u96fe', u'\u7a7f\u8fc7', u'\u524d\u4eba', u'\u683d\u4e0b', u'\u7684', u'\u6811', u'\u4e8e\u662f', u'\u8fdb\u5165', u'\u8ba4\u8f93', u'\u6124\u6012', u'\u6211', u'\u6e05\u695a', u'\u6211', u'\u8d70', u'\u5728', u'\u8fd9\u6761', u'\u4e0d\u5f52\u8def', u'\u4e3a', u'\u6bcf\u4e2a', u'\u97f5\u811a', u'\u800c', u'\u7275\u80a0', u'\u7231', u'\u7684', u'\u4e0d\u662f', u'\u82b1\u9999', u'\u6211', u'\u6df1\u7231\u7740', u'\u8fd9\u7247', u'\u5730\u4e0b', u'\u7eaf\u771f', u'\u7684', u'\u571f\u58e4', u'\u575a\u6301', u'\u7740', u'\u6839', u'\u7684', u'\u4fe1\u4ef0', u'\u4e00\u5982\u65e2\u5f80', u'\u591a\u5c11', u'\u65f6', u'\u6ca1\u4eba', u'\u6b23\u8d4f', u'\u81f3\u5c11', u'\u662f', u'\u5b64\u82b3\u81ea\u8d4f', u'\u5fd8\u8bb0', u'\u6211', u'\u50cf', u'\u4e2a', u'\u5b69\u5b50',

# Token look up

In [15]:
def token_lookup(token, vocab, word_embedding):
    try:
        return [float(x) for x in word_embedding[vocab.index(token)+1].strip().split(' ')[1:]]
    except ValueError:
        print('Token not in vocabulary!')
        # See https://github.com/facebookresearch/fastText#obtaining-word-vectors-for-out-of-vocabulary-words for more details
        return ''

In [16]:
len(token_lookup(vocab[5], wiki_vocab, words))

300

In [8]:
# Skipgram model
model = load_model('zh/wiki.zh.bin')

In [20]:
print(model.get_word_vector(vocab[1]))

[-1.64944708e-01 -2.94708818e-01 -1.20754756e-01 -3.09936672e-01
  6.37601092e-02 -2.40231514e-01 -3.71293902e-01  1.44274905e-01
 -9.92775857e-02  7.42849857e-02  2.17024520e-01  1.55438393e-01
  4.50631678e-01  7.64232650e-02  2.16057256e-01  4.16888416e-01
  8.26665610e-02 -8.16601142e-02 -5.49373738e-02 -1.18559822e-01
  1.29962638e-01  2.90348232e-01 -3.01082432e-01 -1.81621977e-03
 -4.09690052e-01  1.76661704e-02 -2.81581312e-01  3.01812068e-02
  4.43937391e-01  7.44513050e-02 -3.16726595e-01 -3.67642403e-01
 -6.18718080e-02 -1.29553545e-02 -4.01773304e-02 -1.17563143e-01
  1.89177191e-03 -2.51461953e-01  6.98114336e-02  1.27828360e-01
 -1.36616215e-01 -1.81653649e-01 -7.80129433e-02  7.68107846e-02
 -1.58297732e-01  1.67796090e-01 -2.62066036e-01  2.58101169e-02
 -4.64072347e-01  1.38130531e-01 -4.89218570e-02  1.10212512e-01
 -8.34304914e-02 -3.01598728e-01  6.73357910e-03 -1.22148782e-01
 -1.99100763e-01 -5.83374836e-02  4.24957603e-01 -1.08874626e-01
  1.69648170e-01  1.55079

In [21]:
help(model)

Help on instance of _FastText in module fastText.FastText:

class _FastText
 |  This class defines the API to inspect models and should not be used to
 |  create objects. It will be returned by functions such as load_model or
 |  train.
 |  
 |  In general this API assumes to be given only unicode for Python2 and the
 |  Python3 equvalent called str for any string-like arguments. All unicode
 |  strings are then encoded as UTF-8 and fed to the fastText C++ API.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, model=None)
 |  
 |  get_dimension(self)
 |      Get the dimension (size) of a lookup vector (hidden layer).
 |  
 |  get_input_matrix(self)
 |      Get a copy of the full input matrix of a Model. This only
 |      works if the model is not quantized.
 |  
 |  get_input_vector(self, ind)
 |      Given an index, get the corresponding vector of the Input Matrix.
 |  
 |  get_labels(self, include_freq=False)
 |      Get the entire list of labels of the dictionary optionally
 | 

In [22]:
test = [[model.get_word_vector(word) for word in sentence] for sentence in lyrics[:3]]

In [26]:
len(test[2][0])

300